In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive
%ls

/content/drive/MyDrive
'Bobby p3 draft.gdoc'                      zp2019_hw1.pdf
'Bobby P3.gdoc'                            zp2019_hw2.ipynb
'Bobby Peng Farewell .gdoc'                zp2019_hw2.pdf
'Colab Notebooks'/                         zp2019_hw3.pdf
 lazydata/                                'zp2019_hw4 (1).pdf'
 leftImg8bit_trainvaltest/                'zp2019_hw4_part1 (1).py'
 NYU_SR_TUNF.pdf                           zp2019_hw4_part1.py
 NYU_Unofficial_Transcript.pdf            'zp2019_hw4_part2 (1).py'
'P2 Draft Bobby Peng.gdoc'                 zp2019_hw4_part2.py
'P2EX2 (1).gdoc'                           zp2019_hw4.pdf
 P2EX2.gdoc                               'zp2019_hw5 (1).ipynb'
'PRACTICE-UA1 EXAM 1.gdoc'                'zp2019_hw5 (2).ipynb'
'PRACTICE-UA1 EXAM 1.pdf'                 'zp2019_hw5 (3).ipynb'
'Reflective writing Bobby Peng.gdoc'      'zp2019_hw5_extra_credit (1).ipynb'
 Resume_Bobby_Peng.pdf                    'zp2019_hw5_extra_credit (2).ipynb'
'Screen S

In [5]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np
import cv2
import pickle as pkl
import torch.nn.functional as F
import torch.optim as optim

In [6]:
class LazyLoadDataset(Dataset):
  def __init__(self, path, train=True, transform=None):
    self.transform = transform
    path = path + ('train/' if train else 'test/')
    self.pathX = path + 'X/'
    self.pathY = path + 'Y/'
    self.data = os.listdir(self.pathX)

  def __getitem__(self, idx):
    f = self.data[idx]
    img0 = cv2.imread(self.pathX+f+'/rgb/0.png')
    img1 = cv2.imread(self.pathX+f+'/rgb/1.png')
    img2 = cv2.imread(self.pathX+f+'/rgb/2.png')
    if self.transform is not None:
      img0 = self.transform(img0)
      img1 = self.transform(img1)
      img2 = self.transform(img2)
    depth = np.load(self.pathX+f+'/depth.npy')
    field_id = pkl.load(open(self.pathX+f+'/field_id.pkl', 'rb'))
    Y = np.load(self.pathY+f+'.npy')
    return (img0, img1, img2, depth, field_id), Y
  
  def getTestitem(self, idx):
    f = self.data[idx]
    img0 = cv2.imread(self.pathX+f+'/rgb/0.png')
    img1 = cv2.imread(self.pathX+f+'/rgb/1.png')
    img2 = cv2.imread(self.pathX+f+'/rgb/2.png')
    if self.transform is not None:
      img0 = self.transform(img0)
      img1 = self.transform(img1)
      img2 = self.transform(img2)
    depth = np.load(self.pathX+f+'/depth.npy')
    field_id = pkl.load(open(self.pathX+f+'/field_id.pkl', 'rb'))
    return img0, img1, img2, depth, field_id

  def __len__(self):
    return len(self.data)

In [1]:
#find mean and std
'''
train_data = LazyLoadDataset('./lazydata/', transform=transforms.Compose([transforms.ToTensor()]))
train_loader = DataLoader(train_data, batch_size=12, shuffle=True)
train_it = iter(train_loader)
def get_mean_std(loader):
  c=0
  mean0, mean1, mean2, std0, std1, std2, total_image_count= 0, 0, 0, 0, 0, 0, 0
  now = next(train_it)
  while True:
    (img0, img1, img2, depth, field_id), Y = now
    image_count_in_batch= img0.size(0)
    img0 = img0.view(image_count_in_batch, img0.size(1), -1)
    img1 = img1.view(image_count_in_batch, img1.size(1), -1)
    img2 = img2.view(image_count_in_batch, img2.size(1), -1)
    mean0 += img0.mean(2).sum(0)
    mean1 += img1.mean(2).sum(0)
    mean2 += img2.mean(2).sum(0)
    std0 += img0.std(2).sum(0)
    std1 += img1.std(2).sum(0)
    std2 += img2.std(2).sum(0)
    total_image_count += image_count_in_batch
    print(c)
    if c == 282: break
    c+=1
    now = next(train_it)

  mean0 /= total_image_count
  mean1 /= total_image_count
  mean2 /= total_image_count
  std0 /= total_image_count
  std1 /= total_image_count
  std2 /= total_image_count
  return mean0, mean1, mean2, std0, std1, std2

mean0, mean1, mean2, std0, std1, std2 = get_mean_std(train_loader)
total_mean = (mean0+mean1+mean2)/3
total_std = (std0+std1+std2)/3
print(total_mean, total_std)
print(mean0, mean1, mean2, std0, std1, std2)
'''
#tensor([0.4851, 0.4623, 0.4356]) tensor([0.2187, 0.2176, 0.2331])

#tensor([0.4352, 0.4170, 0.3960]) tensor([0.5008, 0.4879, 0.4697]) tensor([0.5193, 0.4820, 0.4412]) 
#tensor([0.1992, 0.1987, 0.2111]) tensor([0.2276, 0.2252, 0.2417]) tensor([0.2293, 0.2288, 0.2465])

"\ntrain_data = LazyLoadDataset('./lazydata/', transform=transforms.Compose([transforms.ToTensor()]))\ntrain_loader = DataLoader(train_data, batch_size=12, shuffle=True)\ntrain_it = iter(train_loader)\ndef get_mean_std(loader):\n  c=0\n  mean0, mean1, mean2, std0, std1, std2, total_image_count= 0, 0, 0, 0, 0, 0, 0\n  now = next(train_it)\n  while True:\n    (img0, img1, img2, depth, field_id), Y = now\n    image_count_in_batch= img0.size(0)\n    img0 = img0.view(image_count_in_batch, img0.size(1), -1)\n    img1 = img1.view(image_count_in_batch, img1.size(1), -1)\n    img2 = img2.view(image_count_in_batch, img2.size(1), -1)\n    mean0 += img0.mean(2).sum(0)\n    mean1 += img1.mean(2).sum(0)\n    mean2 += img2.mean(2).sum(0)\n    std0 += img0.std(2).sum(0)\n    std1 += img1.std(2).sum(0)\n    std2 += img2.std(2).sum(0)\n    total_image_count += image_count_in_batch\n    print(c)\n    if c == 282: break\n    c+=1\n    now = next(train_it)\n\n  mean0 /= total_image_count\n  mean1 /= total_

In [8]:
#define transform(totensor and normalize)
mean, std = [0.4851, 0.4623, 0.4356], [0.2187, 0.2176, 0.2331]
my_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))])

In [10]:
train_data = LazyLoadDataset('./lazydata/', transform=my_transform)
test_data = LazyLoadDataset('./lazydata/', train=False, transform=my_transform)

In [11]:
train_loader = DataLoader(train_data, batch_size=12, shuffle=True)
train_it = iter(train_loader)

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
def train(epoch, model, optimizer):

    model.train()
    for batch_idx, ((img0, img1, img2, depth, field_id), Y) in enumerate(train_loader):
        for batch in range(12):
          i0, i1, i2, d, id, y = img0[batch], img1[batch], img2[batch], depth[batch], field_id[batch], Y[batch]
          data = torch.stack((i0, i1, i2), 0)
          data =data.view(9, 224, -1)
          y = y.view(1, -1)
          torch.mul(y, 1000)

          optimizer.zero_grad()
          output = model(data)
          torch.mul(output, 1000)
          lossfn = nn.MSELoss()
          loss = torch.sqrt(lossfn(output.float(), y.float()))
          loss.backward()
          optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * 12, len(train_data),
                100. * batch_idx * 12 / len(train_data), loss.item()))                                                  

In [17]:
class CNN(nn.Module):
    def __init__(self, input_size, conv_feature, fc_feature, output_size):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(input_size, conv_feature, kernel_size=5)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(conv_feature, conv_feature, kernel_size=5)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.conv3 = nn.Conv2d(conv_feature, conv_feature, kernel_size=5)
        self.LF1 = nn.Linear(conv_feature*49*49, fc_feature)
        self.LF2 = nn.Linear(fc_feature, output_size)
        
    def forward(self, x):
        x = self.maxpool1(F.relu(self.conv1(x)))
        x = self.maxpool2(F.relu(self.conv2(x)))
        x = self.conv3(x)
        x = x.view(-1, 6*49*49)
        x = F.relu(self.LF1(x))
        x = self.LF2(x)
        return x

    #score 0.0073

In [13]:
# Training settings 
conv_features = 6 # number of feature maps
fc_features = 50
output_size = 12

model_cnn = CNN(9, conv_features, fc_features, output_size) # create CNN model
model_cnn.to(device)
optimizer = torch.optim.SGD(model_cnn.parameters(), lr=0.01, momentum=0.5)

In [70]:
for epoch in range(0, 1):
    train(epoch, model_cnn, optimizer)

Train Epoch: 0 [0/3396 (0%)]	Loss: 0.034368
Train Epoch: 0 [120/3396 (4%)]	Loss: 0.015445
Train Epoch: 0 [240/3396 (7%)]	Loss: 0.013365
Train Epoch: 0 [360/3396 (11%)]	Loss: 0.016154
Train Epoch: 0 [480/3396 (14%)]	Loss: 0.015894
Train Epoch: 0 [600/3396 (18%)]	Loss: 0.016623
Train Epoch: 0 [720/3396 (21%)]	Loss: 0.012801
Train Epoch: 0 [840/3396 (25%)]	Loss: 0.011902
Train Epoch: 0 [960/3396 (28%)]	Loss: 0.010370
Train Epoch: 0 [1080/3396 (32%)]	Loss: 0.010873
Train Epoch: 0 [1200/3396 (35%)]	Loss: 0.007087
Train Epoch: 0 [1320/3396 (39%)]	Loss: 0.008341
Train Epoch: 0 [1440/3396 (42%)]	Loss: 0.007633
Train Epoch: 0 [1560/3396 (46%)]	Loss: 0.009321
Train Epoch: 0 [1680/3396 (49%)]	Loss: 0.004584
Train Epoch: 0 [1800/3396 (53%)]	Loss: 0.010414
Train Epoch: 0 [1920/3396 (57%)]	Loss: 0.013201
Train Epoch: 0 [2040/3396 (60%)]	Loss: 0.005483
Train Epoch: 0 [2160/3396 (64%)]	Loss: 0.005895
Train Epoch: 0 [2280/3396 (67%)]	Loss: 0.004108
Train Epoch: 0 [2400/3396 (71%)]	Loss: 0.005704
Train 

In [18]:
# Training settings 
conv_features = 6 # number of feature maps
fc_features = 25
output_size = 12

model_cnn = CNN(9, conv_features, fc_features, output_size) # create CNN model
model_cnn.to(device)
optimizer = torch.optim.SGD(model_cnn.parameters(), lr=0.01, momentum=0.5)
for epoch in range(0, 1):
    train(epoch, model_cnn, optimizer)

Train Epoch: 0 [0/3396 (0%)]	Loss: 0.043899
Train Epoch: 0 [120/3396 (4%)]	Loss: 0.034656
Train Epoch: 0 [240/3396 (7%)]	Loss: 0.015240
Train Epoch: 0 [360/3396 (11%)]	Loss: 0.017372
Train Epoch: 0 [480/3396 (14%)]	Loss: 0.012820
Train Epoch: 0 [600/3396 (18%)]	Loss: 0.016436
Train Epoch: 0 [720/3396 (21%)]	Loss: 0.018251
Train Epoch: 0 [840/3396 (25%)]	Loss: 0.012948
Train Epoch: 0 [960/3396 (28%)]	Loss: 0.011732
Train Epoch: 0 [1080/3396 (32%)]	Loss: 0.013824
Train Epoch: 0 [1200/3396 (35%)]	Loss: 0.015254
Train Epoch: 0 [1320/3396 (39%)]	Loss: 0.009157
Train Epoch: 0 [1440/3396 (42%)]	Loss: 0.008217
Train Epoch: 0 [1560/3396 (46%)]	Loss: 0.006940
Train Epoch: 0 [1680/3396 (49%)]	Loss: 0.007246
Train Epoch: 0 [1800/3396 (53%)]	Loss: 0.010749
Train Epoch: 0 [1920/3396 (57%)]	Loss: 0.004687
Train Epoch: 0 [2040/3396 (60%)]	Loss: 0.010433
Train Epoch: 0 [2160/3396 (64%)]	Loss: 0.007694
Train Epoch: 0 [2280/3396 (67%)]	Loss: 0.009651
Train Epoch: 0 [2400/3396 (71%)]	Loss: 0.005014
Train 

In [81]:
class CNN2(nn.Module):
    def __init__(self, input_size, conv_feature, fc_feature, output_size):
        super(CNN2, self).__init__()
        self.conv1 = nn.Conv2d(input_size, conv_feature, kernel_size=5)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(conv_feature, conv_feature, kernel_size=3)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.conv3 = nn.Conv2d(conv_feature, conv_feature, kernel_size=5)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        self.conv4 = nn.Conv2d(conv_feature, conv_feature, kernel_size=5)
        self.LF1 = nn.Linear(conv_feature*21*21, fc_feature)
        self.LF2 = nn.Linear(fc_feature, output_size)
        
    def forward(self, x):
        x = self.maxpool1(F.relu(self.conv1(x)))
        x = self.maxpool2(F.relu(self.conv2(x)))
        x = self.maxpool3(F.relu(self.conv3(x)))
        x = self.conv4(x)
        x = x.view(-1, 6*21*21)
        x = F.relu(self.LF1(x))
        x = self.LF2(x)
        return x

    #score 0.0073

In [82]:
# Training settings 
conv_features = 6 # number of feature maps
fc_features = 50
output_size = 12

model_cnn2 = CNN2(9, conv_features, fc_features, output_size) # create CNN model
model_cnn2.to(device)
optimizer = torch.optim.SGD(model_cnn2.parameters(), lr=0.01, momentum=0.5)

In [83]:
for epoch in range(0, 1):
    train(epoch, model_cnn2, optimizer)

Train Epoch: 0 [0/3396 (0%)]	Loss: 0.065336
Train Epoch: 0 [120/3396 (4%)]	Loss: 0.013173
Train Epoch: 0 [240/3396 (7%)]	Loss: 0.026738
Train Epoch: 0 [360/3396 (11%)]	Loss: 0.016703
Train Epoch: 0 [480/3396 (14%)]	Loss: 0.018942
Train Epoch: 0 [600/3396 (18%)]	Loss: 0.018432
Train Epoch: 0 [720/3396 (21%)]	Loss: 0.017284
Train Epoch: 0 [840/3396 (25%)]	Loss: 0.011171
Train Epoch: 0 [960/3396 (28%)]	Loss: 0.007906
Train Epoch: 0 [1080/3396 (32%)]	Loss: 0.021231
Train Epoch: 0 [1200/3396 (35%)]	Loss: 0.013534
Train Epoch: 0 [1320/3396 (39%)]	Loss: 0.008936
Train Epoch: 0 [1440/3396 (42%)]	Loss: 0.016263
Train Epoch: 0 [1560/3396 (46%)]	Loss: 0.011361
Train Epoch: 0 [1680/3396 (49%)]	Loss: 0.006430
Train Epoch: 0 [1800/3396 (53%)]	Loss: 0.006261
Train Epoch: 0 [1920/3396 (57%)]	Loss: 0.005378
Train Epoch: 0 [2040/3396 (60%)]	Loss: 0.008962
Train Epoch: 0 [2160/3396 (64%)]	Loss: 0.008382
Train Epoch: 0 [2280/3396 (67%)]	Loss: 0.007077
Train Epoch: 0 [2400/3396 (71%)]	Loss: 0.013980
Train 

In [16]:
class CNN3(nn.Module):
    def __init__(self, input_size, conv_feature, fc_feature, output_size):
        super(CNN3, self).__init__()
        self.conv1 = nn.Conv2d(input_size, conv_feature, kernel_size=9)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(conv_feature, conv_feature, kernel_size=9)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.conv3 = nn.Conv2d(conv_feature, conv_feature, kernel_size=9)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        self.conv4 = nn.Conv2d(conv_feature, conv_feature, kernel_size=9)
        self.LF1 = nn.Linear(conv_feature*13*13, fc_feature)
        self.LF2 = nn.Linear(fc_feature, output_size)
        
    def forward(self, x):
        x = self.maxpool1(F.relu(self.conv1(x)))
        x = self.maxpool2(F.relu(self.conv2(x)))
        x = self.maxpool3(F.relu(self.conv3(x)))
        x = self.conv4(x)
        x = x.view(-1, 6*13*13)
        x = F.relu(self.LF1(x))
        x = self.LF2(x)
        return x

In [18]:
# Training settings 
conv_features = 6 # number of feature maps
fc_features = 50
output_size = 12

model_cnn3 = CNN3(9, conv_features, fc_features, output_size) # create CNN model
model_cnn3.to(device)
optimizer = torch.optim.SGD(model_cnn3.parameters(), lr=0.01, momentum=0.5)

In [19]:
for epoch in range(0, 1):
    train(epoch, model_cnn3, optimizer)

Train Epoch: 0 [0/3396 (0%)]	Loss: 0.059391
Train Epoch: 0 [120/3396 (4%)]	Loss: 0.016135
Train Epoch: 0 [240/3396 (7%)]	Loss: 0.019092
Train Epoch: 0 [360/3396 (11%)]	Loss: 0.018479
Train Epoch: 0 [480/3396 (14%)]	Loss: 0.014485
Train Epoch: 0 [600/3396 (18%)]	Loss: 0.015617
Train Epoch: 0 [720/3396 (21%)]	Loss: 0.013822
Train Epoch: 0 [840/3396 (25%)]	Loss: 0.012890
Train Epoch: 0 [960/3396 (28%)]	Loss: 0.011017
Train Epoch: 0 [1080/3396 (32%)]	Loss: 0.010539
Train Epoch: 0 [1200/3396 (35%)]	Loss: 0.011207
Train Epoch: 0 [1320/3396 (39%)]	Loss: 0.015509
Train Epoch: 0 [1440/3396 (42%)]	Loss: 0.026937
Train Epoch: 0 [1560/3396 (46%)]	Loss: 0.018449
Train Epoch: 0 [1680/3396 (49%)]	Loss: 0.009970
Train Epoch: 0 [1800/3396 (53%)]	Loss: 0.023841
Train Epoch: 0 [1920/3396 (57%)]	Loss: 0.008393
Train Epoch: 0 [2040/3396 (60%)]	Loss: 0.019636
Train Epoch: 0 [2160/3396 (64%)]	Loss: 0.013843
Train Epoch: 0 [2280/3396 (67%)]	Loss: 0.012460
Train Epoch: 0 [2400/3396 (71%)]	Loss: 0.006484
Train 

In [ ]:
class CNN4(nn.Module):
    def __init__(self, input_size, conv_feature, fc_feature, output_size):
        super(CNN3, self).__init__()
        self.conv1 = nn.Conv2d(input_size, conv_feature, kernel_size=9)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(conv_feature, 12, kernel_size=9)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.conv3 = nn.Conv2d(12, 24, kernel_size=9)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        self.conv4 = nn.Conv2d(24, 48, kernel_size=9)
        self.LF1 = nn.Linear(48*13*13, fc_feature)
        self.LF2 = nn.Linear(fc_feature, output_size)
        
    def forward(self, x):
        x = self.maxpool1(F.relu(self.conv1(x)))
        x = self.maxpool2(F.relu(self.conv2(x)))
        x = self.maxpool3(F.relu(self.conv3(x)))
        x = self.conv4(x)
        x = x.view(-1, 48*13*13)
        x = F.relu(self.LF1(x))
        x = self.LF2(x)
        return x

In [ ]:
# Training settings 
conv_features = 6 # number of feature maps
fc_features = 50
output_size = 12

model_cnn4 = CNN3(9, conv_features, fc_features, output_size) # create CNN model
model_cnn4.to(device)
optimizer = torch.optim.SGD(model_cnn4.parameters(), lr=0.01, momentum=0.5)

In [20]:
import csv
import pandas as pd

outfile = 'submission.csv'

titles = ['ID', 'FINGER_POS_1', 'FINGER_POS_2', 'FINGER_POS_3', 'FINGER_POS_4', 'FINGER_POS_5', 'FINGER_POS_6',
         'FINGER_POS_7', 'FINGER_POS_8', 'FINGER_POS_9', 'FINGER_POS_10', 'FINGER_POS_11', 'FINGER_POS_12']
preds = []


model_cnn4.eval()
with open(outfile, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(titles)
    for i in range(len(test_data)):
      if i % 10 == 0: print(i)
      img0, img1, img2, depth, field_id = test_data.getTestitem(i)
      data = torch.stack((img0, img1, img2), 0)
      data =data.view(9, 224, -1)
      output = model_cnn4(data.float())  
      output = output.tolist()
      output = output[0]
      output.insert(0, field_id)
      csvwriter.writerow(output)
print("Written to csv file {}".format(outfile))

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
Written to csv file submission.csv
